<a href="https://colab.research.google.com/github/marendtz/News_NER/blob/master/prodigy_create_custom_datasetdict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparations**

In [1]:
import getpass
#get github access token for collab
token = getpass.getpass()

··········


In [4]:
# clean workdir
%cd /content/
%rm -r /content/*
%ls -la
%pwd

/content
total 16
drwxr-xr-x 1 root root 4096 Apr 30 11:51 ./
drwxr-xr-x 1 root root 4096 Apr 30 11:49 ../
drwxr-xr-x 4 root root 4096 Apr 27 13:34 .config/


'/content'

In [5]:
# clone github repo
!git clone https://{token}@github.com/marendtz/News_NER.git

Cloning into 'News_NER'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 9), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (23/23), 160.86 KiB | 2.09 MiB/s, done.


In [6]:
%pip install -r /content/News_NER/requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 k

**Create huggingface dataset from prodigy annotations.jsonl**

In [18]:
import datasets
from datasets import Features, Sequence, DatasetDict
from datasets.features import Value, ClassLabel

import random

import spacy

from transformers import AutoTokenizer

from huggingface_hub import notebook_login, create_repo



In [11]:
notebook_login()

In [13]:
# helper function to convert offset formatted labels to IOB tag
def convert_spans_to_iob(text, ents):
  tags = ["O"] * len(text)
  for ent in ents:
    start, end, label = ent["token_start"], ent["token_end"], ent["label"]
    if start == end:
      tags[start] = 'B-'+ label
    else:
      tags[start] = 'B-' + label
      tags[start+1: end + 1] = ['I-'+label]*(end - start)
  return tags

In [22]:
# function to convert prodigy info to needed features
def label_and_fill_table(dataset):

  tokens = [word['text'] for word in dataset['tokens']]
  dataset['tokens'] = tokens
  #print("tokens")
  #print(tokens)
  
  dataset['ner_tags_iob'] = convert_spans_to_iob(tokens,dataset['spans'] or [])
  #print('ner_tags_iob')
  #print(dataset['ner_tags_iob'])
  
  return dataset

In [20]:
"""
A class to hold the data and labels for the model.
Initialized from a JSONL file and procesed.
"""

# A helper function that returns True if 'jsonl' is found in a string, otherwise False
def is_jsonl(string):
  if 'jsonl' in string:
    print("is jsonl")
    return True
  else:
    return False

class NERdataset():
  def __init__(self, data_file, seed):
    self.data_file = data_file
    self.seed = seed
    
    # Load dataset
    self.load()
    # Process the data (and save if save is not None)
    self.process()


  # 1) Load using datasets.Dataset.from_json if the data is an unprocessed jsonl file, otherwise load from a preprocessed dataset using load_dataset
  def load(self):
    if is_jsonl(self.data_file):
      self.dataset = datasets.Dataset.from_json(self.data_file)
      print("dataset build from data_file")
    else:
      print("no jsonl supplied as data_file")
    return self

  # 2) Process loaded data to create features needed
  def process_jsonl(self):
    
    print(self.dataset)  
    # create necessary columns from prodigy output + convert offsets to iob format
    self.dataset = self.dataset.map(label_and_fill_table)

    # create information about labels used in the dataset      
    # ensure fixed labeling for transparency (adapted from PAN-X)
    self.label2id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
    self.id2label = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
    self.num_classes = len(self.label2id)
    self.label_list = list(self.label2id.keys())

    # create label colum with info about labels used
    label_column = []
    for sentence in self.dataset["ner_tags_iob"]:
      label_column.append([self.label2id[tag] for tag in sentence])
    self.dataset = self.dataset.add_column("ner_tags", label_column)

    # remove unneccesary columns
    rm_cols = list(set(self.dataset.column_names) - set(['tokens','ner_tags']))      
    self.dataset = self.dataset.map(remove_columns=rm_cols)

    print(self.dataset)
    # adapt features    
    features = Features(
      { 'ner_tags': Sequence(feature=ClassLabel(num_classes=self.num_classes, names=self.label_list, names_file=None, id=None),length=-1, id=None),
        'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
      }
    )    
    self.dataset = self.dataset.map(features.encode_example, features=features)

  # 3) Create DatasetDict that can be loaded to Huggingface for later use in training
  def create_datasetdict(self): 

    # create    
    intermediate_dataset = self.dataset
    # shuffle dataset
    shuffled_dataset = intermediate_dataset.shuffle(seed=self.seed)
    
    # build DataSetDict with train, test, val split
    
    # 90% train, 10% test + validation
    train_testvalid = shuffled_dataset.train_test_split(test_size=0.5)
    # Split the 10% test + valid in half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
    
    self.datasetdict = DatasetDict({
        'train': train_testvalid['train'],
        'test': test_valid['test'],
        'validation': test_valid['train']})
    
    
    # push the tokenized dataset to the hub
    self.datasetdict.push_to_hub(repo_id="maren-hugg/news_ner_dict", private=True)

  # Process the data
  def process(self):
    if is_jsonl(self.data_file):
      self.process_jsonl()
      self.create_datasetdict()  
    else:
      print("data_file is not of format jsonl")
        


In [23]:
# create class object 
dataset_class = NERdataset(
     data_file="/content/News_NER/custom_data/annotations_space.jsonl", 
     seed = 42)

is jsonl
dataset build from data_file
is jsonl
Dataset({
    features: ['text', 'tokens', '_input_hash', '_task_hash', '_view_id', 'spans', 'answer', '_timestamp'],
    num_rows: 4
})


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 4
})


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]